In [1]:
from tensorflow.keras.models import Model, load_model
from load_data import load_dataset
import numpy as np

Using TensorFlow backend.


In [2]:
loaded_model = load_model("Saved_Model/Model.h5")
loaded_model.set_weights(loaded_model.get_weights())
matrix_size = loaded_model.layers[-2].output.shape[1]
new_model = Model(loaded_model.inputs, loaded_model.layers[-2].output)

In [3]:
new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8_input (InputLayer)  [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 122, 122, 64)      3200      
_________________________________________________________________
batch_normalization_12 (Batc (None, 122, 122, 64)      256       
_________________________________________________________________
average_pooling2d_8 (Average (None, 61, 61, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 28, 28, 128)       401536    
_________________________________________________________________
batch_normalization_13 (Batc (None, 28, 28, 128)       512       
_________________________________________________________________
average_pooling2d_9 (Average (None, 14, 14, 128)       0     

In [4]:
images, labels = load_dataset(verbose=1, mode="Test")
images = np.expand_dims(images, axis=3)

Compiling Training and Testing Sets ...


In [5]:
print(images.shape)
print(len(labels))

(300, 128, 128, 1)
300


In [6]:
images = images / 255.

In [7]:
decodeDict = {'151404': 'The Northern Lights', '152103': 'Smile is the key', '152253': 'Milky Way', '152254': 'Mayd Hubb meets Joe Pilgrim - Tribute to Yabby You - Panda meets Jamma Dim REMIX', '152258': 'Purple Trip', '152261': 'Danse Macabre', '152262': 'Die Brücke', '152324': 'Greenhouse On Mars', '152418': 'Snosti ti Dojdov', '152425': 'Opa Tsupa', '152480': 'A Perceptible Shift', '152543': 'BITCHEZ', '152545': 'Where the Sunshine Goes', '152568': 'Everything Dances', '152569': 'The Magic of Bamboo', '152570': 'Collapse Into Innocence', '153337': 'The Vent', '153383': 'La Samba (en public)', '153452': 'Métal Province', '153946': "Hey, that's not normal", '153955': 'Not Coming Home', '153956': 'Border Collie', '154303': 'In It to Win It', '154305': 'Calm Flight', '154306': 'Yummy', '154307': '12:01 AM', '154308': 'MIA', '154309': 'A1 Symphony', '154413': 'Do Easy', '154414': 'Dead Can Dance (uncensored)'}

In [8]:
for i in np.unique(labels):
    print(decodeDict[i])

The Northern Lights
Smile is the key
Milky Way
Mayd Hubb meets Joe Pilgrim - Tribute to Yabby You - Panda meets Jamma Dim REMIX
Purple Trip
Danse Macabre
Die Brücke
Greenhouse On Mars
Snosti ti Dojdov
Opa Tsupa
A Perceptible Shift
BITCHEZ
Where the Sunshine Goes
Everything Dances
The Magic of Bamboo
Collapse Into Innocence
The Vent
La Samba (en public)
Métal Province
Hey, that's not normal
Not Coming Home
Border Collie
In It to Win It
Calm Flight
Yummy
12:01 AM
MIA
A1 Symphony
Do Easy
Dead Can Dance (uncensored)


In [9]:
decodeDictReverse = {x: y for y, x in decodeDict.items()}
recommend_wrt = input("Enter Song name:\n")
recommend_wrt = decodeDictReverse[recommend_wrt]

Enter Song name:
Do Easy


In [11]:
prediction_anchor = np.zeros((1, matrix_size))
count = 0
predictions_song = []
predictions_label = []
counts = []
distance_array = []

# Calculate the latent feature vectors for all the songs.
for i in range(0, len(labels)):
    if labels[i] == recommend_wrt:
        test_image = images[i]
        test_image = np.expand_dims(test_image, axis=0)
        prediction = new_model.predict(test_image)
        prediction_anchor = prediction_anchor + prediction
        count = count + 1
    elif labels[i] not in predictions_label:
        predictions_label.append(labels[i])
        test_image = images[i]
        test_image = np.expand_dims(test_image, axis=0)
        prediction = new_model.predict(test_image)
        predictions_song.append(prediction)
        counts.append(1)
    elif labels[i] in predictions_label:
        index = predictions_label.index(labels[i])
        test_image = images[i]
        test_image = np.expand_dims(test_image, axis=0)
        prediction = new_model.predict(test_image)
        predictions_song[index] = predictions_song[index] + prediction
        counts[index] = counts[index] + 1
# Count is used for averaging the latent feature vectors.
prediction_anchor = prediction_anchor / count
for i in range(len(predictions_song)):
    predictions_song[i] = predictions_song[i] / counts[i]
    # Cosine Similarity - Computes a similarity score of all songs with respect
    # to the anchor song.
    distance_array.append(np.sum(prediction_anchor * predictions_song[i]) / (np.sqrt(np.sum(prediction_anchor**2)) * np.sqrt(np.sum(predictions_song[i]**2))))

distance_array = np.array(distance_array)
recommendations = 0

print("Recommendation is:")

# Number of Recommendations is set to 2.
while recommendations < 2:
    index = np.argmax(distance_array)
    value = distance_array[index]
    print("Song Name: " + decodeDict[predictions_label[index]] + " with value = %f" % value)
    distance_array[index] = -np.inf
    recommendations = recommendations + 1

Recommendation is:
Song Name: Everything Dances with value = 0.926041
Song Name: MIA with value = 0.876097
